In [ ]:
!pip install --upgrade pip


In [ ]:
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Classe dados 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py

!pip install kats
from kats.consts import TimeSeriesData
from kats.models.prophet import ProphetModel, ProphetParams
import pandas as pd
import time

# Autenticações google sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe

Forecast Suporte N1 

In [ ]:
"""
# Importar base histórica
base_1 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arquivos/dados_forecast/bases/tickets_criados_tickets_recebidos_1.csv')
base_2 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arquivos/dados_forecast/bases/tickets_criados_tickets_recebidos_2.csv')
base_3 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arquivos/dados_forecast/bases/tickets_criados_tickets_recebidos_3.csv')
base_4 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/arquivos/dados_forecast/bases/tickets_criados_tickets_recebidos_4.csv')
base_tt = pd.concat([base_1,
                     base_2,
                     base_3,
                     base_4])

# renomear colunas
colunas = {'ID do ticket':'ID_do_ticket',
'Assunto':'Assunto',
'Status':'Status',
'Nome do grupo':'Nome_do_grupo',
'Data de criação':'Data_de_criacao',
'Resolved date':'Data_resolucao',
'Tag nome':'Tag_nome'
}


#Renomear colunas
base_tt.rename(columns=colunas, inplace=True)
base_tt['Data_de_criacao'] = base_tt['Data_de_criacao'].str.replace('AM','').str.replace('PM','').str.strip()
ajuste_data =  base_tt['Data_de_criacao'].str.split(" ", n = 1, expand = True)
base_tt['Data_de_criacao'] = ajuste_data[0]
base_tt['Data_de_criacao'] = pd.to_datetime(base_tt['Data_de_criacao'],infer_datetime_format=True,format='%y%m%d')
base_tt['Data_resolucao'] = base_tt['Data_resolucao'].str.replace('AM','').str.replace('PM','')
ajuste_data =  base_tt['Data_resolucao'].str.split(" ", n = 1, expand = True)
base_tt['Data_resolucao'] = ajuste_data[0]
base_tt['Data_resolucao'] = pd.to_datetime(base_tt['Data_resolucao'],infer_datetime_format=True,format='%y%m%d')


base_tt_2022 = base_tt.query('Data_de_criacao>="2022-01-01"') # Base 2022
de_para_forecast = dados.busca_tabela_drive('forecast_colab', 'de_para_tickets') # Base de_para Suporte N1
base_tt_2022_depara = pd.merge(base_tt_2022,de_para_forecast, how='left', on='Nome_do_grupo') # Merge entre as tabelas
base_tt_2022_suporte = base_tt_2022_depara.query('Supervisor!=""') # Selecionar somente suporte n1



# Inicio Forecast Tickets
base_tt_2022_suporte['Supervisor'] = base_tt_2022_suporte['Supervisor'].apply(str)
lista_supervisores = sorted(set(base_tt_2022_suporte['Supervisor'].to_list()))
lista_supervisores.remove('nan')
#lista_supervisor.remove('')
base_forecast = base_tt_2022_suporte[['Data_de_criacao','Data_resolucao','ID_do_ticket', 'Nome_do_grupo', 'Supervisor']]
base_forecast['Qtd'] = 1

for super in lista_supervisores:
    # Ajuste DataFrame
    base_tt_grupo = base_forecast.query('Supervisor=="{}"'.format(super))
    base_realizado = pd.pivot_table(base_tt_grupo, index='Data_de_criacao', values='Qtd', aggfunc='sum')
    base_realizado.reset_index(inplace=True)
    base_realizado['fcst'] = 0
    base_realizado['fcst_lower'] = 0
    base_realizado['fcst_upper'] = 0
    # Calculo Forecast  
    base_realizado['Data_de_criacao'] = pd.to_datetime(base_realizado['Data_de_criacao'],infer_datetime_format=True,format='%y%m%d')
    #base_realizado.rename(columns={'ID_do_ticket':'Qtd'}, inplace=True)
    df_s = TimeSeriesData(time=base_realizado['Data_de_criacao'], value=base_realizado['Qtd'])
    params = ProphetParams(seasonality_mode='multiplicative')
    model = ProphetModel(df_s, params)
    model.fit()
    forecast = model.predict(steps=30)
    forecast.rename(columns={'time':'Data_de_criacao'}, inplace=True)
    forecast['Qtd'] = 0
    # Merge Realizado + Forecast
    forecast_final = pd.concat([base_realizado, forecast])
    forecast_final = forecast_final.astype({"fcst": int, "fcst_lower": int,"fcst_upper": int})
    forecast_final['Supervisores'] = super
    def linhas():
        arquivo = gc.open('forecast_colab')
        # abrir planilha 'aba'
        aba = arquivo.worksheet('check_forecast_tickets')
        # colocar a aba dentro de um data_frame
        df1 = pd.DataFrame(aba.get_all_records())
        #Visualização
        numero = df1['Data_de_criacao'].count()+2
        n = int(numero)
        return n
     
    def input_drive():
        #Abrir planilha e primeira aba
        time.sleep(3)
        pagina = gc.open('forecast_colab').worksheet('check_forecast_tickets')
        # Limpar base
        #pagina.clear()
        # Importar dataframe na planilha
        return set_with_dataframe(pagina, forecast_final, row=linhas(), include_column_header=False)
    input_drive()
    time.sleep(3)
    print('Calculando forecast do supervisor {}!'.format(super))
"""

Forecast - Whats

In [ ]:
# Importar base
base_original_whats = dados.busca_tabela_drive('Dash_Whatsapp', 'Histórico')
base_original_whats_v2 = base_original_whats[['Initiated At', 'Conversation id', 'Group name']]
# Renomer colunas
colunas = {'Initiated At':'Data_criacao',
           'Conversation id':'Conversas',
           'Group name':'Nome_do_grupo'}
base_original_whats_v2.rename(columns=colunas, inplace=True)
# Ajuste data
ajuste_data =  base_original_whats_v2['Data_criacao'].str.split(" ", n = 1, expand = True)
base_original_whats_v2['Data_criacao'] = ajuste_data[0]

# Base de_para
depara_whats = dados.busca_tabela_drive('forecast_colab','de_para_whatsapp')
# Merge 
base_original_whats_v3 = pd.merge(base_original_whats_v2, depara_whats, how='left', on='Nome_do_grupo')


# Inicio Forecast Whats
base_original_whats_v3['Supervisor'] = base_original_whats_v3['Supervisor'].apply(str)
lista_supervisores_whats = sorted(set(base_original_whats_v3['Supervisor'].to_list()))
lista_supervisores_whats.remove('nan')
#lista_supervisor.remove('')
base_forecast_whats = base_original_whats_v3[['Data_criacao','Conversas', 'Nome_do_grupo', 'Supervisor']]
base_forecast_whats['Qtd'] = 1

for super in lista_supervisores_whats:
    # Ajuste DataFrame
    base_tt_grupo = base_forecast_whats.query('Supervisor=="{}"'.format(super))
    base_realizado = pd.pivot_table(base_tt_grupo, index='Data_criacao', values='Qtd', aggfunc='sum')
    base_realizado.reset_index(inplace=True)
    base_realizado['fcst'] = 0
    base_realizado['fcst_lower'] = 0
    base_realizado['fcst_upper'] = 0
    # Calculo Forecast  
    base_realizado['Data_criacao'] = pd.to_datetime(base_realizado['Data_criacao'],infer_datetime_format=True,format='%y%m%d')
    #base_realizado.rename(columns={'ID_do_ticket':'Qtd'}, inplace=True)
    df_s = TimeSeriesData(time=base_realizado['Data_criacao'], value=base_realizado['Qtd'])
    params = ProphetParams(seasonality_mode='multiplicative')
    model = ProphetModel(df_s, params)
    model.fit()
    #informar dias para pegar futuro
    forecast = model.predict(steps=32)v
    forecast.rename(columns={'time':'Data_criacao'}, inplace=True)
    forecast['Qtd'] = 0
    # Merge Realizado + Forecast
    forecast_final = pd.concat([base_realizado, forecast])
    forecast_final = forecast_final.astype({"fcst": int, "fcst_lower": int,"fcst_upper": int})
    forecast_final['Supervisores'] = super
    def linhas():
        arquivo = gc.open('forecast_colab')
        # abrir planilha 'aba'
        aba = arquivo.worksheet('check_forecast_whats')
        # colocar a aba dentro de um data_frame
        df1 = pd.DataFrame(aba.get_all_records())
        #Visualização
        numero = df1['Data_de_criacao'].count()+2
        n = int(numero)
        return n
     
    def input_drive():
        #Abrir planilha e primeira aba
        pagina = gc.open('forecast_colab').worksheet('check_forecast_whats')
        # Limpar base
        #pagina.clear()
        # Importar dataframe na planilha
        return set_with_dataframe(pagina, forecast_final, row=linhas(), include_column_header=False)
    input_drive()
    print('Calculando forecast do supervisor {}!'.format(super))




In [ ]:
analise_hoh = base_original_whats[['Initiated At', 'Conversation id', 'Group name']]